In [ ]:
#@title Autoreload
%load_ext autoreload
%autoreload 2

## Python Art Segmentation Tools
Segment an image of a painting or drawing into layers, and generate an SVG "bundle" that contains paths of each segment.

### Step 1

Clone the repo and run the setup script to install dependencies


In [ ]:
!git clone https://github.com/carlmoore256/art-segmentation-tools
%cd /content/art-segmentation-tools
!sh setup.sh

## Step 2
Imports

In [ ]:
%cd /content/art-segmentation-tools
from segmentation import load_model
from image import Image, alpha_blend_images
from mask import Mask, AnnotationMask
from segmented_image import SegmentedImage
from segment_anything import SamAutomaticMaskGenerator
from product import export_bundle
# import sys
# sys.path.append("/content/art-segmentation-tools/deepsvg")

## Step 3
Load the segmentation model from [Segment Anything](https://github.com/facebookresearch/segment-anything), and setup the mask generator with some segmentation parameters.

In [ ]:
SEGMENTATION_MODEL = load_model()

From Segment Anything's [notebook](https://github.com/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb) on automatic mask segmentation:

> "There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes."

[Source for SamAutomaticMaskGenerator](https://github.com/facebookresearch/segment-anything/blob/6fdee8f2727f4506cfbbe553e23b895e27956588/segment_anything/automatic_mask_generator.py)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(
    model=SEGMENTATION_MODEL,
    points_per_side=32,# (32) Number of points to be sampled along one side of the image. The total number of points is points_per_side**2. If None, 'point_grids' must provide explicit point sampling.
    points_per_batch=64, # (64) Sets the number of points run simultaneously by the model. Higher numbers may be faster but use more GPU memory
    pred_iou_thresh=0.85, # (0.88) A filtering threshold in [0,1], using the model's predicted mask quality
    stability_score_thresh=0.92, # (0.95) A filtering threshold in [0,1], using the stability of the mask under changes to the cutoff used to binarize the model's mask predictions.
    stability_score_offset=1.0, # (1.0) The amount to shift the cutoff when calculated the stability score
    box_nms_thresh=0.7, # (0.7) The box IoU cutoff used by non-maximal suppression to filter duplicate masks.
    crop_n_layers=1, # (0) If > 0, mask prediction will be run again on crops of the image. Sets the number of layers to run, where each layer has 2**i_layer number of image crops.
    crop_nms_thresh=0.7, # (0.7) The box IoU cutoff used by non-maximal suppression to filter duplicate masks between different crops.
    crop_overlap_ratio= 512 / 1500, # (512/1500, ~0.3413) Sets the degree to which crops overlap. In the first crop layer, crops will overlap by this fraction of the image length. Later layers with more crops scale down this overlap.
    crop_n_points_downscale_factor=2, # (1) The number of points-per-side sampled in layer n is scaled down by crop_n_points_downscale_factor**n.
    point_grids=None, # (None) A list over explicit grids of points used for sampling, normalized to [0,1]. The nth grid in the list is used in the nth crop layer. Exclusive with points_per_side
    min_mask_region_area=100, # (0) If > 0, postprocessing will be applied to remove disconnected regions and holes in masks with area smaller than min_mask_region_area. Requires opencv.
    output_mode="binary_mask" # ("binary_mask")  The form masks are returned in. Can be 'binary_mask', 'uncompressed_rle', or 'coco_rle'. 'coco_rle' requires pycocotools. For large resolutions, 'binary_mask' may consume large amounts of memory.
)

# default mask generator
# mask_generator = SamAutomaticMaskGenerator(SEGMENTATION_MODEL)

## Step 4
Load an image into the custom Image object included in this repo. Some notable features include:
- `Image.resize(tuple)`
- `Image.pad_to_square()`
- `Image.show()`

Images need to be padded to squares to work correctly with the tracing (this should be fixed in the future). Images should also be a reasonable size to not exceed the maximum GPU memory.

In [ ]:
# image_path = "/content/drive/MyDrive/Mario/PROJECT WITH CARL/ALL ART/THE MIRACLE.jpg"
image_path = "/content/CandyLandEdited.jpg"
image = Image(image_path)
print(f"Loaded image with dimensions: {image.shape}")
image.pad_to_square()
image.resize((2500,2500))
image.show()

## Step 5
Run the image segmentation. The Image class works together with Mask and SegmentedImage to generate an object that contains a set of masked layers.

- `SegmentedImage` is constructed with an `Image` object
- `SegmentedImage.segment(SamAutomaticMaskGenerator)` runs the segmentation and generates the layers

If segmentation is taking an indefinite amount of time, cancel it and tweak the segmentation parameters. The default parameters tend to be fast

In [ ]:
seg_image = SegmentedImage(image)
seg_image.segment(mask_generator)
print(seg_image)

### Visualize Results

WARNING: This is currently really inefficient and takes a LONG time

In [ ]:
masks_image = seg_image.visualize_masks(0.7)
masks_image.show(figsize=(5,5))

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

print(f'Getting Masks')
masks = seg_image.get_masks_by_area()

@interact(mask_idx=(0,len(masks)))
def visualize_mask(mask_idx=0):
  print(f'{mask_idx}')
  masks[mask_idx].show(title=f'Mask {mask_idx}')

Show the "background," which is any area that has not been segmented

In [ ]:
seg_image.get_background().show((5,5))

## Step 6
Export the bundled SVG, which will trace the paths into SVG paths, simplify the paths, and write the image to the background layer. It will also write another image layer if you choose to use inpainting.

In [ ]:
export_bundle(seg_image, "BastionPebble", "/content/drive/MyDrive/Mario/SVGBundles")

## Step 7 (Optional)
Do some inpainting on the image - provide the image and mask, and it will fill in the rest. This can be cool if the final product is a parallaxed SVG, where when the background is revealed, it reveals more imagery that looks like a background, rather than a hole, or a copy of the image piece above it. 

- WARNING: This takes a lot of GPU Memory since there are two models loaded in. If this is running on colab, the session may crash.

First, we can try to create a new background by inpainting any of the areas leftover from the segmentation

In [ ]:
from inpainting import inpaint_image
from utils import dilate_mask

You can choose to dilate the mask or not. Dilation will ensure the edges extend out further than the original, so it has more coverage.

In [ ]:
# uncomment one or the other
mask = Mask(dilate_mask(seg_image.unmasked_area_mask().invert().get_writeable_data(), 3))
# mask = seg_image.unmasked_area_mask().invert()

Run the inpainting with a prompt, providing whatever mask you want as the area to inpaint

In [ ]:
prompt = "an empty field"

result_image = inpaint_image(seg_image.get_background(), mask, prompt)
result_image.show(figsize=(8,8))

In [ ]:
result_image.save("/content/drive/MyDrive/Mario/Generative/meet-me-at-shasta-bg-2.png")

If this is satisfying for a background, we can set it as the background for our SegmentedImage
- SementedImage uses the background (.get_background()) to construct the final output image

In [ ]:
seg_image.set_background(result_image)

Convert into an SVG with transparent layers

In [ ]:
from product import export_bundle

export_bundle(seg_image, "NecessaryStare", "/content/drive/MyDrive/Mario/SVGBundles")